<a href="https://colab.research.google.com/github/niz11/Bachelor_Thesis_Colab_Files/blob/main/train_autoencoder_dots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint

data = np.load('drive/My Drive/moving_dots.npy')

trainng_size = 98000
test_size = 2000
train_loop = 4900
test_loop = 100


input_train = np.empty((trainng_size,64,64,1),  dtype=np.uint8)
input_test = np.empty((test_size,64,64,1),  dtype=np.uint8)

index = 0
for i in range(train_loop):
    for j in range(20):
        input_train[index] = data[i][j]
        index +=1

index = 0            
for i in range(test_loop):
    for j in range(10):
        input_test[index] = data[i][j]
        index +=1

print(data.shape)
print(input_train.shape)
print(input_test.shape)

In [ ]:
import matplotlib.pyplot as plt
# Test By visualisation 
n = 20
randomSmaple = np.random.randint(750, size=1)
x = randomSmaple * 20
print(x)
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    if i < 10:
      plt.imshow(input_train[i + x].reshape(64, 64))
    else:
      plt.imshow(input_train[i + x].reshape(64, 64))

    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# Data & model configuration
img_width, img_height = 64, 64
batch_size = 64
epochs = 5
latent_dim = 64
num_channels = 1

input_shape = (img_height, img_width, num_channels)

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# # =================
# # Encoder
# # =================

# Definition
inputs       = Input(shape=input_shape, name='encoder_input')
encodingLayers  = Conv2D(filters=128, kernel_size=5, strides=1, padding='same', activation='relu')(inputs)
encodingLayers  = BatchNormalization()(encodingLayers)
encodingLayers  = Conv2D(filters=256, kernel_size=5, strides=2, padding='same', activation='relu')(encodingLayers)
encodingLayers  = BatchNormalization()(encodingLayers)
encodingLayers  = Conv2D(filters=512, kernel_size=5, strides=1, padding='same', activation='relu')(encodingLayers)
encodingLayers  = BatchNormalization()(encodingLayers)
encodingLayers  = Conv2D(filters=1024, kernel_size=5, strides=2, padding='same', activation='relu')(encodingLayers)
encodingLayers  = BatchNormalization()(encodingLayers)
output       = Flatten()(encodingLayers)
output       = Dense(latent_dim, activation='relu')(output)
output       = BatchNormalization()(output)

# Get Conv2D shape for Conv2DTranspose operation in decoder
conv_shape = K.int_shape(encodingLayers)

# Instantiate encoder
encoder = Model(inputs, output, name='encoder')
encoder.summary()

# =================
# Decoder
# =================

# Definition
decoder_input = Input(shape=(latent_dim, ), name='decoder_input')
decoder_inputs  = Dense(conv_shape[1] * conv_shape[2] * conv_shape[3], activation='relu')(decoder_input)
decoder_inputs  = BatchNormalization()(decoder_inputs)
decoder_inputs  = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(decoder_inputs)
decoder_laters = Conv2DTranspose(filters=1024, kernel_size=5, strides=1, padding='same', activation='relu')(decoder_inputs)
decoder_laters = BatchNormalization()(decoder_laters)
decoder_laters = Conv2DTranspose(filters=512, kernel_size=5, strides=2, padding='same', activation='relu')(decoder_laters)
decoder_laters = BatchNormalization()(decoder_laters)
decoder_laters = Conv2DTranspose(filters=256, kernel_size=5, strides=1, padding='same', activation='relu')(decoder_laters)
decoder_laters = BatchNormalization()(decoder_laters)
decoder_laters = Conv2DTranspose(filters=128, kernel_size=5, strides=2, padding='same', activation='relu')(decoder_laters)
decoder_laters = BatchNormalization()(decoder_laters)
decoder_output     = Conv2DTranspose(filters=num_channels, kernel_size=3, activation='sigmoid', padding='same', name='decoder_output')(decoder_laters)

# Instantiate decoder
decoder = Model(decoder_input, decoder_output, name='decoder')
decoder.summary()

# Instantiate VAE
model_outputs = decoder(encoder(inputs))
model = Model(inputs, model_outputs, name='model')
model.summary()

# checkpoint
checkpoint_path = "drive/My Drive/thesis/dot_encoder/save/cp-{epoch:04d}.ckpt"
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 period=1,
                                                 verbose=1)

model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(input_train, input_train, epochs = epochs, batch_size = batch_size, validation_split = 0.2, callbacks=[cp_callback])


In [ ]:
encdoing = encoder.predict(input_test)
print(encdoing.shape)
decoded = decoder.predict(encdoing)
print(decoded.shape)

In [ ]:
import matplotlib.pyplot as plt
for j in range(10,15):
  n = 20
  # randomSmaple = np.random.randint(4, size=1)
  x = j * 20

  plt.figure(figsize=(20, 4))
  for i in range(n):
      # display original
      ax = plt.subplot(2, n, i + 1)
      if i < 10:
        plt.imshow(decoded[i+ x].reshape(64, 64))
      else:
        plt.imshow(decoded[i+ x].reshape(64, 64))

      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
  plt.show()

  plt.figure(figsize=(20, 4))
  for i in range(n):
      # display original
      ax = plt.subplot(2, n, i + 1)
      if i < 10:
        plt.imshow(input_test[i+ x].reshape(64, 64))
      else:
        plt.imshow(input_test[i+ x].reshape(64, 64))

      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
  plt.show()

In [ ]:
# serialize model to JSON
model_json = encoder.to_json()
with open("encoderDots2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
encoder.save_weights("encoderDots2.h5")
print("Saved model to disk")

# serialize model to JSON
model_json = decoder.to_json()
with open("decoderDots2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
decoder.save_weights("decoderDots2.h5")
print("Saved model to disk")